In [15]:
from selenium import webdriver
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import TimeoutException
import time
import re
from bs4 import BeautifulSoup
import requests
from collections import defaultdict
import pickle

In [ ]:
sites_to_scrape = ["https://www.instagram.com/explore/locations/6889842/paris-france/",
                   "https://www.instagram.com/explore/locations/213385402/london-united-kingdom/",
                   "https://www.instagram.com/explore/locations/118001404/copenhagen-denmark/",
                   "https://www.instagram.com/explore/locations/212988663/new-york-new-york/",
                   "https://www.instagram.com/explore/locations/116231/amsterdam-netherlands/",
                   "https://www.instagram.com/explore/locations/274029466/singapore/"]

scroll = 350
allLinks = set()
userStore = defaultdict(list)

path_to_chromedriver = 'C:\\Users\\njacobsen\\Documents\\Chrome Driver\\chromedriver.exe'
driver = webdriver.Chrome(executable_path = path_to_chromedriver)

In [ ]:
driver.get(sites_to_scrape[-1])

for i in range(scroll):
    try:
        driver.execute_script("window.scrollBy(0,500)")
    except:
        pass
    time.sleep(2)
    scrapeLinks = [x.get_attribute('href') for x in driver.find_elements_by_css_selector('a')]
    cleanLinks = [x for x in scrapeLinks if x != None and "/p/" in x]
    allLinks.update(cleanLinks)

pickle.dump( allLinks, open( "singapore.p", "wb" ) )

In [ ]:
allLinks = pickle.load( open( "paris.p", "rb" ) )

In [ ]:
userStore = defaultdict(list)

for i in list(allLinks):
    driver.get(i)
    try:
        likes = re.sub("\D","",driver.find_elements_by_class_name("_nzn1h")[0].text)
    except:
        likes = 0

    soup = BeautifulSoup(driver.page_source, "lxml")
    try:
        userName = [x.get_attribute('href') for x in driver.find_elements_by_css_selector('a')][0][26:-1]
    except:
        continue
    try:
        image_list = [] # create empty list to save img urls
        for link in soup.find_all('img'): # find img tags in HTML
            m = re.search('src="(.+?)"', str(link))
            image_list.append(m.group(1)) # append url path to list
    except:
        continue
    try:
        img = image_list[1]
        userStore[userName].append([img, "copenhagen", likes])
    except:
        pass
    

In [ ]:
paris = pickle.load( open( "paris.p", "rb" ) )
amsterdam = pickle.load( open( "amsterdam.p", "rb" ) )
copenhagen = pickle.load( open( "copenhagen.p", "rb" ) )
newyork = pickle.load( open( "newyork.p", "rb" ) )
london = pickle.load( open( "london.p", "rb" ) )
singapore = pickle.load( open( "singapore.p", "rb" ) )

allLinks = list(paris)+list(amsterdam)+list(copenhagen)+list(newyork)+list(london)+list(singapore)
allLinks = set(allLinks)

In [ ]:
userNames = []

In [ ]:
for i in list(allLinks):
    driver.get(i)
    try:
        name = driver.find_elements_by_class_name("_eeohz")[0].text
    except:
        continue
    userNames.append(name)

In [ ]:
uniqueUsers = set(userNames)

In [ ]:
uniqueUsers = pickle.load( open( "uniqueUsers.p", "rb" ) )

In [58]:
uniqueUsers = list(pickle.load( open( "uniqueUsers.p", "rb" ) ))

path_to_chromedriver = 'C:\\Users\\njacobsen\\Documents\\Chrome Driver\\chromedriver.exe'
driver = webdriver.Chrome(executable_path = path_to_chromedriver)
userStore = defaultdict(list)
scroll = 7

for user in uniqueUsers[0:100]:
    userLinks = set()
    driver.get("https://www.instagram.com/"+user)

    for i in range(scroll):
        try:
            driver.execute_script("window.scrollBy(0,500)")
        except:
            pass
        time.sleep(2)

        scrapeLinks = [x.get_attribute('href') for x in driver.find_elements_by_css_selector('a')]
        cleanLinks = [x for x in scrapeLinks if x != None and "/p/" in x]
        userLinks.update(cleanLinks)
        
    for link in userLinks:
        driver.get(link)
        soup = BeautifulSoup(driver.page_source, "lxml")
        try:
            place = driver.find_elements_by_class_name("_6y8ij")[0].text
        except:
            place = "no place"
            
        try:
            likes = re.sub("\D","",driver.find_elements_by_class_name("_nzn1h")[0].text)
        except:
            likes = 0
            
        try:
            image_list = [] # create empty list to save img urls
            for stuff in soup.find_all('img'): # find img tags in HTML
                m = re.search('src="(.+?)"', str(stuff))
                image_list.append(m.group(1)) # append url path to list
            img = image_list[1]
        except:
            pass
        userStore[user].append([link, place, likes, img])

KeyboardInterrupt: 

In [59]:
pickle.dump(userStore,open("userStore.p", "wb"))

In [61]:
len(userStore.keys())

77

In [51]:
len(userStore['ingridivorra'])

36

In [57]:
userStore['mcangeluk']

[['https://www.instagram.com/p/Bg1Kre5gRV1/?taken-by=mcangeluk',
  'no place',
  '52',
  'https://scontent-arn2-1.cdninstagram.com/vp/f64124074d212792bc0578b1e660037f/5B9BC86A/t51.2885-15/e35/29417102_240315593182134_4953292989708369920_n.jpg'],
 ['https://www.instagram.com/p/BhKBYPnAF2U/?taken-by=mcangeluk',
  'London, United Kingdom',
  '80',
  'https://scontent-arn2-1.cdninstagram.com/vp/ae10971488d20ec37924637beb12cdef/5B9BBCEA/t51.2885-15/e35/29739165_2023011527938540_3925264624353542144_n.jpg'],
 ['https://www.instagram.com/p/BgpBEq6gTFR/?taken-by=mcangeluk',
  'The Shard London',
  '61',
  'https://scontent-arn2-1.cdninstagram.com/vp/0a98b13c9a633c3b88752332e1f69ce8/5B60E461/t51.2885-15/e35/29094427_252414571968508_9031024932980523008_n.jpg'],
 ['https://www.instagram.com/p/Bhj-jpNB17a/?taken-by=mcangeluk',
  'no place',
  '48',
  'https://scontent-arn2-1.cdninstagram.com/vp/f1bca19622efeaadc761b54bd67c3393/5B64A227/t51.2885-15/e35/30589875_441658606262752_3660940469963063296_n.

In [55]:
len(userStore['bbasjtr_singapore'])

36

In [56]:
userStore['bbasjtr_singapore']

[['https://www.instagram.com/p/BheTS4MlJnI/?taken-by=bbasjtr_singapore',
  'Singapore',
  '974',
  'https://scontent-arn2-1.cdninstagram.com/vp/0452c0fdd2e8fa6c38430c95220a201b/5B648DBC/t51.2885-15/e35/29737977_208379079927148_7673436026127253504_n.jpg'],
 ['https://www.instagram.com/p/BhTd_tLnmx5/?taken-by=bbasjtr_singapore',
  'Singapore',
  0,
  'https://scontent-arn2-1.cdninstagram.com/vp/aadb9dd9d2fb06f3ba1905fb82a04b7b/5ADE37F8/t51.2885-15/e15/29714559_172013913451272_1446645994333667328_n.jpg'],
 ['https://www.instagram.com/p/BhWhjmQHZQD/?taken-by=bbasjtr_singapore',
  'Singapore',
  '618',
  'https://scontent-arn2-1.cdninstagram.com/vp/03294dd1c16c8bbcf26355dba7d09b93/5ADE5306/t51.2885-15/e15/30085096_389787064871565_7162838111976685568_n.jpg'],
 ['https://www.instagram.com/p/BhbbzfalI26/?taken-by=bbasjtr_singapore',
  'Singapore',
  0,
  'https://scontent-arn2-1.cdninstagram.com/vp/aba79c3359a56db8f664e980016dad0e/5ADE2506/t51.2885-15/e15/29737778_189964078475950_2329050774803